<a href="https://colab.research.google.com/github/Yuehua-Liu/Taiwan-memory-electronic-resource-crawler/blob/main/%E5%8F%B0%E7%81%A3%E8%A8%98%E6%86%B6_%E9%9B%BB%E5%AD%90%E6%96%87%E7%8D%BB%E7%88%AC%E8%9F%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step1. 輸入要下載的台灣記憶電子書網址

---



1.   替換「url」參數為自己要的網址
2.   注意：程式會檢查網址有效性，「tm.ncl.edu.tw/article?」開頭的網址才會通過驗證
3.   程式會列出抓到的頁面清單





### 執行我下載必要套件

---
匯出 PDF 的必要工具


In [27]:
# 下載必要套件
!pip install img2pdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.9 MB/s eta 0:00:00
  Created wheel for img2pdf: filename=img2pdf-0.5.1-py3-none-any.whl size=49275 sha256=50ba6cc0cdf85f9f90c81a9f541e26b9ee996fd798c0b2a5465cdeec20ae6e79
  Stored in directory: /root/.cache/pip/wheels/b7/e2/32/e12ea533072415219fc91a248cc413f34c505881ba850bf058
Successfully built img2pdf


### 輸入下載網址、檔案命名等自定義資訊

In [28]:
# #############################【使用者自行填寫區】###################################
# 要爬取的網址請貼在這邊
url = "https://tm.ncl.edu.tw/article?u=008_001_0000345457&lang=chn"

# 儲存資料夾命名
folder_name = "test"

# PDF 檔案命名(不用加附檔名)
pdf_name = "隨便一本電子書名"
# ####################################################################################

### 開始解析網站

In [29]:
import os
import time
import shutil
import img2pdf
import requests
from bs4 import BeautifulSoup


print("SYS: 程式啟動...")
# 建立儲存資料夾
folder_name = "/content/drive/MyDrive/" + folder_name
domain = "https://tm.ncl.edu.tw"
if "https://tm.ncl.edu.tw/article?" in url:
    print("SYS: 有效網址，開始搜尋文檔...")
    response = requests.get(url)
    if response.status_code == 200:
        print("SYS: 搜尋成功，開始解析網站分頁項目...")
        soup = BeautifulSoup(response.text, "html.parser")
        # 抓XPATH: //*[@id="modal-pic-list"]/div/div/div[2]
        target = soup.find("table", {"class": "table table-hover table-responsive"})
        urls = target.find_all("a")
        hrefs = []
        for url in urls:
            hrefs.append([url.getText(), domain + url.get("href")])
        print(f"SYS：總共抓到 '{len(hrefs)}' 個頁面可供下載")
        print("________________________________________________________________________________________")
        if len(hrefs) > 5:
            print("SYS：資料太多，只顯示前5個頁面...")
            for href in hrefs[:5]:
                print(f"SYS：{href[0]}) ==> {href[1]}")
        else:
            for href in hrefs:
                print(f"SYS：{href[0]}) ==> {href[1]}")
    else:
        print("SYS: 連線失敗，請排除問題")
else:
    print("SYS: 無效網址，請確認網址有效性")

SYS: 程式啟動...
SYS: 有效網址，開始搜尋文檔...
SYS: 搜尋成功，開始解析網站分頁項目...
SYS：總共抓到 '446' 個頁面可供下載
________________________________________________________________________________________
SYS：資料太多，只顯示前5個頁面...
SYS：a0000001.jpg) ==> https://tm.ncl.edu.tw/TM_DO/008/100632826/000475639/a0000001_watered_watered_72dpi.jpg
SYS：a0000002.jpg) ==> https://tm.ncl.edu.tw/TM_DO/008/100632826/000475639/a0000002_watered_watered_72dpi.jpg
SYS：a0000003.jpg) ==> https://tm.ncl.edu.tw/TM_DO/008/100632826/000475639/a0000003_watered_watered_72dpi.jpg
SYS：a0000004.jpg) ==> https://tm.ncl.edu.tw/TM_DO/008/100632826/000475639/a0000004_watered_watered_72dpi.jpg
SYS：a0000005.jpg) ==> https://tm.ncl.edu.tw/TM_DO/008/100632826/000475639/a0000005_watered_watered_72dpi.jpg


## Step2. 設定儲存路徑，並開始爬取所有頁面

---


0.   (重要)先點選左邊「掛接雲端硬碟」，再執行程式，下載結果才會存到自己雲端硬碟中
1.   確認設定好資料夾名稱後，開始執行程式，便會開始下載分頁檔，因為一筆一筆下載，需要等待久一點
2.   下載結果 可以在左邊資料夾圖示看到



In [24]:
# 確認雲端硬碟是否掛接
if not os.path.exists("/content/drive"):
    print("SYS: Error!! 請先掛接雲端硬碟...")
else:
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    # 建立分頁資料夾
    sub_dir = folder_name + "/pages_files"
    if not os.path.exists(sub_dir):
        os.makedirs(sub_dir)
    # 先清空下載檔案夾，獲取 sub_dir 中的所有檔案和目錄
    items = os.listdir(sub_dir)
    if items:
        # 遍歷並刪除每個檔案或目錄
        for item in items:
            item_path = os.path.join(sub_dir, item)
            try:
                if os.path.isfile(item_path) or os.path.islink(item_path):
                    os.unlink(item_path)  # 刪除檔案或符號連結
                elif os.path.isdir(item_path):
                    shutil.rmtree(item_path)  # 刪除目錄及其所有內容
            except Exception as e:
                print(f'無法刪除 {item_path}。原因: {e}')

    # 開始下載分頁資訊
    for index, href in enumerate(hrefs):
        target_url = href[1]
        f_name = f"{index}_{href[0]}"

        # 開始下載分頁資訊
        print(f"SYS: 開始下載第 {index+1}/{len(hrefs)} 項")
        print(href[1])
        response = requests.get(target_url)
        if response.status_code == 200:
            # 下載檔案
            with open(f"{sub_dir}/{f_name}", "wb") as f:
                f.write(response.content)
            print(f"SYS: 檔案【{f_name}】下載成功")
        else:
            print(f"SYS: 連線失敗，請排除問題")
        print("________________________________________________________________________________________")
        if index == len(hrefs) - 1:
            print("SYS：所有檔案下載完成！")


SYS: 開始下載第 1/446 項
https://tm.ncl.edu.tw/TM_DO/008/100632826/000475639/a0000001_watered_watered_72dpi.jpg
SYS: 檔案【0_a0000001.jpg】下載成功
________________________________________________________________________________________
SYS: 開始下載第 2/446 項
https://tm.ncl.edu.tw/TM_DO/008/100632826/000475639/a0000002_watered_watered_72dpi.jpg
SYS: 檔案【1_a0000002.jpg】下載成功
________________________________________________________________________________________
SYS: 開始下載第 3/446 項
https://tm.ncl.edu.tw/TM_DO/008/100632826/000475639/a0000003_watered_watered_72dpi.jpg
SYS: 檔案【2_a0000003.jpg】下載成功
________________________________________________________________________________________
SYS: 開始下載第 4/446 項
https://tm.ncl.edu.tw/TM_DO/008/100632826/000475639/a0000004_watered_watered_72dpi.jpg
SYS: 檔案【3_a0000004.jpg】下載成功
________________________________________________________________________________________
SYS: 開始下載第 5/446 項
https://tm.ncl.edu.tw/TM_DO/008/100632826/000475639/a0000005_watered_watered_72dpi.jpg
SY

KeyboardInterrupt: 

## Step3. 將下載結果匯出成 PDF 檔

---



1.   先確認「page_files/」資料夾內已經有下載好的分頁
2.   這段程式會將「page_files/」內按照檔名前綴編號彙總成一份PDF檔



In [45]:
# 現在時間戳
current_time = int(time.time())
output_pdf_name = f"{pdf_name}_{current_time}.pdf"
pdf_path = os.path.join(folder_name, output_pdf_name)

# 將 sub_dir 內容，彙總成一份PDF
img_files = [os.path.join(sub_dir, file) for file in os.listdir(sub_dir) if file.endswith((".jpg", ".jpeg", ".gif", ".png", ".tiff", ".JPG", ".JPEG", ".GIF", ".PNG", ".TIFF"))]

# 檢查是否有圖檔
if not img_files:
    print("SYS: 目錄中沒有找到圖檔")
else:
    print(f"SYS: 開始彙總 PDF 檔，共 {len(img_files)} 頁...")
    # 使用 with open 寫入 PDF 文件
    with open(pdf_path, "wb") as f:
        f.write(img2pdf.convert(img_files))
    print(f"SYS: 已成功將圖檔合併成 => {pdf_path}")


SYS: 開始彙總 PDF 檔，共 16 頁...
SYS: 已成功將圖檔合併成 => 隨便一本電子書名_1727318735.pdf
